# Movie Rating Prediction with Python

Regression model to predict movie ratings using features such as genre, director, and actors.
Dataset: IMDb India Movies (Kaggle)

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

## 2. Load Dataset

In [2]:
df = pd.read_csv('IMDb_India_Movies.csv', encoding='latin1')
df.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


## 3. Data Cleaning & Preprocessing

In [3]:
# Drop rows without ratings
df = df.dropna(subset=['Rating'])

# Select required columns
features = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3', 'Year', 'Duration']
df = df[features + ['Rating']]

# Clean Year column: (2019) -> 2019
df['Year'] = df['Year'].str.replace('[()]', '', regex=True)

# Clean Duration column: "109 min" -> 109
df['Duration'] = df['Duration'].str.replace(' min', '', regex=False)

# Convert to numeric
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')

# Drop rows where conversion failed
df = df.dropna()

# Split features and target
X = df.drop('Rating', axis=1)
y = df['Rating']


## 4. Feature Encoding

In [4]:
categorical_features = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']
numeric_features = ['Year', 'Duration']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numeric_features)
    ],
    sparse_threshold=0
)


## 5. Train-Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## 6. Model Training

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        n_jobs=-1
    ))
])

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre', 'Director',
                                                   'Actor 1', 'Actor 2',
                                                   'Actor 3']),
                                                 ('num', 'passthrough',
                                                  ['Year', 'Duration'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

## 7. Model Evaluation

In [7]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MAE:', mae)
print('MSE:', mse)
print('R2 Score:', r2)

MAE: 0.9468374558303886
MSE: 1.526029644876325
R2 Score: 0.1758958384630419


## 8. Prediction on New Movie Data

In [8]:
new_movie = pd.DataFrame({
    'Genre': ['Drama'],
    'Director': ['Unknown'],
    'Actor 1': ['Unknown'],
    'Actor 2': ['Unknown'],
    'Actor 3': ['Unknown'],
    'Year': [2022],
    'Duration': [120]
})

predicted_rating = model.predict(new_movie)
print("Predicted Movie Rating:", predicted_rating[0])

Predicted Movie Rating: 7.871999999999999
